#Lectura librerias

In [ ]:
#Instalación de paquetes
!pip install pyLDAvis -qq
!pip install -qq -U gensim
!pip install spacy -qq
!pip install matplotlib -qq
!pip install seaborn -qq
!python -m spacy download en_core_web_md -qq
import pandas as pd
pd.set_option('display.max_columns', None)
from pandas_profiling import ProfileReport
from pandas.api.types import is_string_dtype, is_numeric_dtype,is_object_dtype
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spacy
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()# Visualise inside a notebook
import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

# Tratamiento de datos
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes

# Gráficos
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import plotly.express as px
from matplotlib import gridspec
from matplotlib.pyplot import *
import seaborn as sns

# Preprocesado y análisis
import statsmodels.api as sm

#import pingouin as pg
from scipy import stats
from scipy.stats import pearsonr

# Preprocesado y modelado
from sklearn import tree
from sklearn.metrics import accuracy_score,auc,make_scorer,confusion_matrix
from sklearn.metrics import plot_confusion_matrix,f1_score,precision_score,recall_score,roc_curve
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Librerías para procesamiento de texto (text mining)
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import sys  

#!{sys.executable} -m pip install contractions
!pip install spacy
import spacy
!python -m spacy download en_core_web_sm

! pip install textblob 
from textblob import TextBlob

#! pip install wordcloud #generador de nube de palabras
from wordcloud import WordCloud, STOPWORDS

#word embeddings
! pip install -U gensim
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel
from gensim.models import Word2Vec
from scipy.spatial.distance import cdist

#Visualizar topic modelling
#import pyLDAvis
#import pyLDAvis.gensim

#Guardado del modelo
import pickle 

#Otras acciones
import itertools
from itertools import chain
from collections import Counter
import scipy as sp
from tqdm.autonotebook import tqdm #barra de progreso
tqdm.pandas()
import scipy.stats as ss
import random

#Figures inline and set visualization style
%matplotlib inline
sns.set()

# Configuración de advertencias(warnings)
import warnings
warnings.filterwarnings('ignore')

# Configuración matplotlib
plt.style.use('ggplot')

# Configuración warnings
import warnings
warnings.filterwarnings('ignore')

#Lectura bases de datos

In [2]:
ConsumerComplaints = pd.read_csv("/content/drive/MyDrive/Tesis/Consumer_Complaints.csv", decimal=',')

In [3]:
complaints = pd.read_csv("/content/drive/MyDrive/Tesis/complaints.csv", decimal=',')

In [4]:
complaints2 = pd.read_csv("/content/drive/MyDrive/Tesis/complaints-2.csv", decimal=',')

In [ ]:
ConsumerComplaints["Complaint ID"].count()

In [ ]:
complaints["Complaint ID"].count()

In [ ]:
complaints2["Complaint ID"].count()

In [8]:
Quejas = complaints.append(complaints2)

In [9]:
Quejas = Quejas.drop_duplicates()

In [ ]:
Quejas["Complaint ID"].count()

In [ ]:
Quejas.shape

In [ ]:
Quejas.dtypes

In [ ]:
Quejas['Date received'] = pd.to_datetime(Quejas['Date received'], errors='coerce')
Quejas['Date sent to company'] = pd.to_datetime(Quejas['Date sent to company'], errors='coerce')
Quejas.dtypes

In [14]:
import datetime

In [ ]:
Quejas['Year'] = Quejas['Date received'].dt.year 
Quejas.head()

In [ ]:
in_2022 = Quejas['Year'] == 2022
in_2022.head()

In [17]:
Quejas2022 = Quejas[in_2022]

In [ ]:
Quejas2022['Year'].unique()

In [ ]:
Quejas2022['Complaint ID'].count()

In [20]:
Quejas_in_years = Quejas[Quejas.Year.isin([2021, 2022])]

In [ ]:
Quejas_in_years['Complaint ID'].count()

In [ ]:
datos = Quejas_in_years[Quejas_in_years['Consumer complaint narrative'].notna()]
datos['Complaint ID'].count()

In [ ]:
df = datos['Consumer complaint narrative'].head(100000)
df.head()

In [24]:
import string 

In [ ]:
df = df.apply(lambda x: x.replace('[{}]'.format(string.punctuation), ''))
df.head()

#Analisis descriptivo

In [ ]:
Quejas_in_years.columns

In [ ]:
Quejas_in_years['Product'].describe()

In [ ]:
# tabla de frecuencia de product
pd.value_counts(Quejas_in_years['Product'])

In [ ]:
# tabla de frecuencia de product %
product_freq = 100 * Quejas_in_years['Product'].value_counts() / len(Quejas_in_years['Product'])
product_freq

In [ ]:
product_freq.plot(kind='barh', title='product %')

In [ ]:
### Lets look at complaints distribution by product type
fig,ax = plt.subplots(figsize=(24,6))
Quejas_in_years['Product'].value_counts().plot(kind = 'barh',title = 'Complaints By Product')
plt.xlabel("Product")
plt.ylabel("Number of complaints")

In [72]:
import matplotlib.pyplot as plt

def show_donut_plot(col):
    
    rating_data = Quejas_in_years.groupby(col)[['Complaint ID']].count().head(10)
    plt.figure(figsize = (16, 12))
    plt.pie(rating_data[['Complaint ID']], autopct = '%1.0f%%', startangle = 140, pctdistance = 1.1, shadow = True)

    # create a center circle for more aesthetics to make it better
    gap = plt.Circle((0, 0), 0.5, fc = 'white')
    fig = plt.gcf()
    fig.gca().add_artist(gap)
    
    plt.axis('equal')
    
    cols = []
    for index, row in rating_data.iterrows():
        cols.append(index)
    plt.legend(cols)
    
    plt.title('PQRS por producto' , loc='center')
    
    plt.show()

In [ ]:
show_donut_plot('Product')

In [ ]:
### Company Distribution by number of complaints
Quejas_in_years['Company'].value_counts()[0:25].plot(kind= 'bar',title = ' Top 25 compañias con PQRS',figsize = (12, 8))
plt.xlabel("Company Name")
plt.ylabel("Number of Complaints")

In [ ]:
# tabla de frecuencia de Company public response
pd.value_counts(Quejas_in_years['Company public response'])

In [ ]:
# tabla de frecuencia de Company public response %
Company_public_response_freq = 100 * Quejas_in_years['Company public response'].value_counts() / len(Quejas_in_years['Company public response'])
Company_public_response_freq

In [ ]:
### Company Distribution by number of complaints
Quejas_in_years['Company public response'].value_counts()[0:25].plot(kind= 'barh',title = ' Tipo de respuesta',figsize = (12, 8))
plt.ylabel("Responses type")
plt.xlabel("Number of responses")

In [ ]:
Company_public_response_freq.plot(kind='barh', title='Company public response %')

In [ ]:
# tabla de frecuencia de Date received
pd.value_counts(Quejas_in_years['Date received'])

In [ ]:
# tabla de frecuencia de Date received %
Date_received_freq = 100 * Quejas_in_years['Date received'].value_counts() / len(Quejas_in_years['Date received'])
Date_received_freq

In [ ]:
Date_received_freq.plot(kind='barh', title='Date received %')

In [ ]:
# tabla de frecuencia de Subproduct
pd.value_counts(Quejas_in_years['Sub-product'])

In [ ]:
# tabla de frecuencia de Subproduct %
Subproduct_freq = 100 * Quejas_in_years['Sub-product'].value_counts() / len(Quejas_in_years['Sub-product'])
Subproduct_freq

In [ ]:
Subproduct_freq.plot(kind='barh', title='Sub-product %')

In [ ]:
# tabla de frecuencia de Issue
pd.value_counts(Quejas_in_years['Issue'])

In [ ]:
# tabla de frecuencia de Issue %
Issue_freq = 100 * Quejas_in_years['Issue'].value_counts() / len(Quejas_in_years['Issue'])
Issue_freq.head(10)

In [ ]:
Issue_freq.head(10).plot(kind='barh', title='Porcentaje de solicitudes agrupadas por problema',figsize=(10,6))

In [ ]:
# tabla de frecuencia de Sub-issue
pd.value_counts(Quejas_in_years['Sub-issue'])

In [ ]:
# tabla de frecuencia de Sub-issue %
Sub_issue_freq = 100 * Quejas_in_years['Sub-issue'].value_counts() / len(Quejas_in_years['Sub-issue'])
Sub_issue_freq

In [ ]:
Subproduct_freq.plot(kind='barh', title='Sub-issue %')

In [ ]:
# tabla de frecuencia de Company
pd.value_counts(Quejas_in_years['Company'])

In [ ]:
# tabla de frecuencia de Company %
Company_freq = 100 * Quejas_in_years['Company'].value_counts() / len(Quejas_in_years['Company'])
Company_freq

In [ ]:
Company_freq.plot(kind='barh', title='Company %')

In [ ]:
# tabla de frecuencia de State
pd.value_counts(Quejas_in_years['State']).head(10)

In [ ]:
# tabla de frecuencia de State %
State_freq = 100 * (Quejas_in_years['State'].value_counts() / len(Quejas_in_years['State'])).head(10)
State_freq

In [ ]:
State_freq.plot(kind='barh', title='Porcentaje de quejas por estado',figsize = (14, 10))

In [ ]:
# tabla de frecuencia de ZIP code
pd.value_counts(Quejas_in_years['ZIP code'])

In [ ]:
# tabla de frecuencia de ZIP code %
ZIP_code_freq = 100 * Quejas_in_years['ZIP code'].value_counts() / len(Quejas_in_years['ZIP code'])
ZIP_code_freq

In [ ]:
ZIP_code_freq.plot(kind='barh', title='ZIP_code %')

In [ ]:
# tabla de frecuencia de Tags
pd.value_counts(Quejas_in_years['Tags'])

In [ ]:
# tabla de frecuencia de Tags %
Tags_freq = 100 * Quejas_in_years['Tags'].value_counts() / len(Quejas_in_years['Tags'])
Tags_freq

In [ ]:
Tags_freq.plot(kind='barh', title='Tags %')

In [ ]:
# tabla de frecuencia de Consumer consent provided?
pd.value_counts(Quejas_in_years['Consumer consent provided?'])

In [ ]:
# tabla de frecuencia de Consumer consent provided? %
Consumer_consent_provided_freq = 100 * Quejas_in_years['Consumer consent provided?'].value_counts() / len(Quejas_in_years['Consumer consent provided?'])
Consumer_consent_provided_freq

In [ ]:
Consumer_consent_provided_freq.plot(kind='barh', title='Consumer consent provided? %')

In [ ]:
# tabla de frecuencia de Submitted via
pd.value_counts(Quejas_in_years['Submitted via'])

In [ ]:
# tabla de frecuencia de Submitted via %
Submitted_via_freq = 100 * Quejas_in_years['Submitted via'].value_counts() / len(Quejas_in_years['Submitted via'])
Submitted_via_freq

In [ ]:
Submitted_via_freq.plot(kind='barh', title='Submitted via %')

In [56]:
import matplotlib.pyplot as plt

def show_donut_plot(col):
    
    rating_data = Quejas_in_years.groupby(col)[['Complaint ID']].count().head(10)
    plt.figure(figsize = (16, 12))
    plt.pie(rating_data[['Complaint ID']], autopct = '%1.0f%%', startangle = 140, pctdistance = 1.1, shadow = True)

    # create a center circle for more aesthetics to make it better
    gap = plt.Circle((0, 0), 0.5, fc = 'white')
    fig = plt.gcf()
    fig.gca().add_artist(gap)
    
    plt.axis('equal')
    
    cols = []
    for index, row in rating_data.iterrows():
        cols.append(index)
    plt.legend(cols)
    
    plt.title('PQRS por canal de envío' , loc='center')
    
    plt.show()

In [ ]:
show_donut_plot('Submitted via')

In [ ]:
# tabla de frecuencia de Company response to consumer
pd.value_counts(Quejas_in_years['Company response to consumer'])

In [ ]:
# tabla de frecuencia de Company response to consumer %
Company_response_consumer_freq = 100 * Quejas_in_years['Company response to consumer'].value_counts() / len(Quejas_in_years['Company response to consumer'])
Company_response_consumer_freq

In [ ]:
Company_response_consumer_freq.plot(kind='barh', title='Company response to consumer %')

In [26]:
import matplotlib.pyplot as plt

def show_donut_plot(col):
    
    rating_data = Quejas_in_years.groupby(col)[['Complaint ID']].count().head(10)
    plt.figure(figsize = (16, 12))
    plt.pie(rating_data[['Complaint ID']], autopct = '%1.0f%%', startangle = 140, pctdistance = 1.1, shadow = True)

    # create a center circle for more aesthetics to make it better
    gap = plt.Circle((0, 0), 0.5, fc = 'white')
    fig = plt.gcf()
    fig.gca().add_artist(gap)
    
    plt.axis('equal')
    
    cols = []
    for index, row in rating_data.iterrows():
        cols.append(index)
    plt.legend(cols)
    
    plt.title('Estado de la PQRS' , loc='center')
    
    plt.show()

In [ ]:
show_donut_plot('Company response to consumer')

In [ ]:
# tabla de frecuencia de Timely response?
pd.value_counts(Quejas_in_years['Timely response?'])

In [ ]:
# tabla de frecuencia de Timely response? %
Timely_response_freq = 100 * Quejas_in_years['Timely response?'].value_counts() / len(Quejas_in_years['Timely response?'])
Timely_response_freq

In [ ]:
Timely_response_freq.plot(kind='bar', title='Respuesta oportuna')

In [57]:
# tabla de frecuencia de Consumer disputed?
pd.value_counts(Quejas_in_years['Consumer disputed?'])

Series([], Name: Consumer disputed?, dtype: int64)

In [ ]:
# tabla de frecuencia de Year
pd.value_counts(Quejas_in_years['Year'])

In [ ]:
# tabla de frecuencia de Year %
Year_freq = 100 * Quejas_in_years['Year'].value_counts() / len(Quejas_in_years['Year'])
Year_freq

In [ ]:
Year_freq.plot(kind='barh', title='Year %')

In [ ]:
#Se excluyen las quejas para revisar correlación entre las demás variables
Datos_in_years=Quejas_in_years[['Company']]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Tabla de contingencia Submitted via / Company response to consumer
pd.crosstab(index=Quejas_in_years['Company response to consumer'],
            columns=Quejas_in_years['Submitted via'], margins=True)

In [ ]:
# tabla de contingencia en porcentajes relativos total
pd.crosstab(index=Quejas_in_years['Company response to consumer'], columns=Quejas_in_years['Submitted via'],
            margins=True).apply(lambda r: r/len(Quejas_in_years) *100,
                                axis=1)

In [ ]:
# tabla de contingencia en porcentajes relativos segun Company response to consumer
pd.crosstab(index=Quejas_in_years['Company response to consumer'], columns=Quejas_in_years['Submitted via']
           ).apply(lambda r: r/r.sum() *100,
                                axis=1)

In [ ]:
# tabla de contingencia en porcentajes relativos segun Submitted via
pd.crosstab(index=Quejas_in_years['Company response to consumer'], columns=Quejas_in_years['Submitted via']
           ).apply(lambda r: r/r.sum() *100,
                                axis=0)

In [ ]:
# Gráfico de barras de sobreviviviente segun clase
plot = pd.crosstab(index=Quejas_in_years['Submitted via'],
            columns=Quejas_in_years['Company response to consumer']).apply(lambda r: r/r.sum() *100,
                                              axis=1).plot(kind='bar')

#Nube de palabras

In [ ]:
!python -m spacy download en_core_web_md -qq

In [ ]:
!python -m spacy download es_core_news_md -qq

In [ ]:
!python -m spacy download en_core_web_sm -qq

In [29]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt


nlp_te=df.tolist()

plt.figure(figsize=(25, 20))
wordcloud = WordCloud(width=800, height=400,background_color="white", max_words=500, stopwords=['XX', 'XXXX','Hi','was','be','this','is','an','am','in',
                                                                                                'are','these','it','the','that','to','On','not','My','by',
                                                                                                'wo','Whom','any','name','dtype','or','as','of','he','one',
                                                                                                'me','will','were','if','with','did','when','she','and',
                                                                                                'they','for','you','so','your','have','been','has','from',
                                                                                                'there','no'],
                      colormap='winter').generate(' '.join(nlp_te))

plt.imshow(wordcloud)
plt.axis("off")
plt.show()

#Modelo de clusterización


In [ ]:
# Install
!pip install pyLDAvis -qq
!pip install -qq -U gensim
!pip install spacy -qq
!pip install matplotlib -qq
!pip install seaborn -qq
!python -m spacy download en_core_web_md -qq

# Import
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spacy
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()# Visualise inside a notebook
import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

In [57]:
# Our spaCy model:
nlp = en_core_web_md.load()
# Tags I want to remove from the text ####REVISAR ESTA LÍNEA PARA SABER LO QUE DEBO TOKENIZAR
removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']

common_words=['xxxx','XX','XXXX','xxxxxxxx']

tokens = []
for pqr in nlp.pipe(df):
   proj_tok = [token.lemma_.lower() 
   for token in pqr 
   if token.pos_ not in removal and not token.is_stop and token.is_alpha and token.lemma_ not in common_words]
   tokens.append(proj_tok)


In [ ]:
from spacy.tokens import token
# Add tokens to new column
df['tokens'] = tokens
df['tokens']

In [59]:
dictionary = Dictionary(df['tokens'])

In [60]:
# Filter dictionary
dictionary.filter_extremes(no_below=10, no_above=0.5, keep_n=200)

# Create corpus
corpus = [dictionary.doc2bow(doc) for doc in df['tokens']]

# LDA model building
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=50, num_topics=10, workers = 4, passes=10)

In [ ]:
# Coherence score using C_umass:
topics = []
score = []
for i in range(1,20,1):
   lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=10, num_topics=i, workers = 4, passes=10, random_state=100)
   cm = CoherenceModel(model=lda_model, corpus=corpus, dictionary=dictionary, coherence='u_mass')
   topics.append(i)
   score.append(cm.get_coherence())
_=plt.plot(topics, score)
_=plt.xlabel('Number of Topics')

_=plt.ylabel('Coherence Score')
plt.show()


In [ ]:
# Coherence score using C_v:
topics = []
score = []
for i in range(1,20,1):
   lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=10, num_topics=i, workers = 4, passes=10, random_state=100)
   cm = CoherenceModel(model=lda_model, texts = df['tokens'], corpus=corpus, dictionary=dictionary, coherence='c_v')
   topics.append(i)
   score.append(cm.get_coherence())
_=plt.plot(topics, score)
_=plt.xlabel('Number of Topics')
_=plt.ylabel('Coherence Score')
plt.show()

In [ ]:
# Optimal model
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=100, num_topics=5, workers = 4, passes=100)

# Print topics
lda_model.print_topics(-1)

# Where does a text belong to
lda_model[corpus][0]
df

# Visualize topics
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_display)